<div class="alert alert-block" style="border: 1px solid #FFB300;background-color:#F9FBE7;">
<font size="3em" style="font-weight:bold;color:#3f8dbf;">  </font><br>
    
### 주요 질문(탐색하고자 하는 질문 리스트)
- 얼마나 많은 고객이 있는가?
- 고객은 어디에 주로 사는가?
- 고객은 주로 어떤 지불방법을 사용하는가?
- 평균 거래액은 얼마일까?
- 일별, 주별, 월별 판매 트렌드는?
- 어떤 카테고리가 가장 많은 상품이 팔렸을까?
- **평균 배송시간은?**
</div>

In [1]:

import pandas as pd

customers = pd.read_csv('00_data/olist_customers_dataset.csv', encoding = 'utf-8-sig')
geolocations = pd.read_csv('00_data/olist_geolocation_dataset.csv', encoding = 'utf-8-sig')
items = pd.read_csv('00_data/olist_order_items_dataset.csv', encoding = 'utf-8-sig')
payments = pd.read_csv('00_data/olist_order_payments_dataset.csv', encoding = 'utf-8-sig')
reviews = pd.read_csv('00_data/olist_order_reviews_dataset.csv', encoding = 'utf-8-sig')
orders = pd.read_csv('00_data/olist_orders_dataset.csv', encoding = 'utf-8-sig')
sellers = pd.read_csv('00_data/olist_sellers_dataset.csv', encoding = 'utf-8-sig')
products = pd.read_csv('00_data/olist_products_dataset.csv', encoding = 'utf-8-sig')

In [2]:
orders_deliver_time = orders[['order_id', 'order_status', 'order_approved_at', 'order_delivered_customer_date', 'order_estimated_delivery_date']].copy()
orders_deliver_time = orders_deliver_time.dropna()
orders_deliver_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96462 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       96462 non-null  object
 1   order_status                   96462 non-null  object
 2   order_approved_at              96462 non-null  object
 3   order_delivered_customer_date  96462 non-null  object
 4   order_estimated_delivery_date  96462 non-null  object
dtypes: object(5)
memory usage: 4.4+ MB


In [3]:
orders_deliver_time['order_approved_at'] = pd.to_datetime(orders_deliver_time['order_approved_at'], format = '%Y-%m-%d %H:%M:%S', errors = 'raise')
#                                             to_
orders_deliver_time['order_delivered_customer_date'] = pd.to_datetime(orders_deliver_time['order_delivered_customer_date'], format = '%Y-%m-%d %H:%M:%S', errors = 'raise')
orders_deliver_time['order_estimated_delivery_date'] = pd.to_datetime(orders_deliver_time['order_estimated_delivery_date'], format = '%Y-%m-%d %H:%M:%S', errors = 'raise')

In [4]:
orders_deliver_time['real_delivery_time'] = orders_deliver_time['order_delivered_customer_date'] - orders_deliver_time['order_approved_at']
orders_deliver_time['estimatedl_delivery_time'] = orders_deliver_time['order_estimated_delivery_date'] - orders_deliver_time['order_approved_at']
orders_deliver_time['real_estimated_differences'] = orders_deliver_time['real_delivery_time'] - orders_deliver_time['estimatedl_delivery_time']
orders_deliver_time.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 96462 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype          
---  ------                         --------------  -----          
 0   order_id                       96462 non-null  object         
 1   order_status                   96462 non-null  object         
 2   order_approved_at              96462 non-null  datetime64[ns] 
 3   order_delivered_customer_date  96462 non-null  datetime64[ns] 
 4   order_estimated_delivery_date  96462 non-null  datetime64[ns] 
 5   real_delivery_time             96462 non-null  timedelta64[ns]
 6   estimatedl_delivery_time       96462 non-null  timedelta64[ns]
 7   real_estimated_differences     96462 non-null  timedelta64[ns]
dtypes: datetime64[ns](3), object(2), timedelta64[ns](3)
memory usage: 6.6+ MB


---
- 특이값 handling

In [6]:
orders_deliver_time['real_delivery_time'].quantile(0.95)

Timedelta('28 days 22:14:08.399999999')

In [7]:
orders_deliver_time['real_delivery_time'].quantile(0.90)

Timedelta('22 days 13:54:40.300000003')

In [8]:
orders_deliver_time = orders_deliver_time[orders_deliver_time['real_delivery_time'] < orders_deliver_time['real_delivery_time'].quantile(.95)]

- 월별 평균 배송시간

In [9]:
orders_deliver_time.head()

,order_id,order_status,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,real_delivery_time,estimatedl_delivery_time,real_estimated_differences
0,e481f51cbdc54678b7cc49136f2d6af7,delivered,2017-10-02 11:07:15,2017-10-10 21:25:13,2017-10-18,8 days 10:17:58,15 days 12:52:45,-8 days +21:25:13
1,53cdb2fc8bc7dce0b6741e2150273451,delivered,2018-07-26 03:24:27,2018-08-07 15:27:45,2018-08-13,12 days 12:03:18,17 days 20:35:33,-6 days +15:27:45
2,47770eb9100c2d0c44946d9cf07ec65d,delivered,2018-08-08 08:55:23,2018-08-17 18:06:29,2018-09-04,9 days 09:11:06,26 days 15:04:37,-18 days +18:06:29
3,949d5b44dbf5de918fe9c16f97b45f8a,delivered,2017-11-18 19:45:59,2017-12-02 00:28:42,2017-12-15,13 days 04:42:43,26 days 04:14:01,-13 days +00:28:42
4,ad21c59c0840e6cb83a9ceb5573f8159,delivered,2018-02-13 22:20:29,2018-02-16 18:17:02,2018-02-26,2 days 19:56:33,12 days 01:39:31,-10 days +18:17:02


In [10]:
orders_deliver_time.groupby('order_approved_at').agg({'real_delivery_time':'mean'})

DataError: No numeric types to aggregate

- DataError: No numeric types to aggregate
- 이래서 데이터 타입을 바꾼거구나

In [17]:
orders_deliver_time_visual = orders_deliver_time[['order_approved_at', 'real_delivery_time']].copy()
orders_deliver_time_visual['real_delivery_time'] = orders_deliver_time_visual['real_delivery_time'].dt.days
orders_deliver_time_visual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91638 entries, 0 to 99440
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_approved_at   91638 non-null  datetime64[ns]
 1   real_delivery_time  91638 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 2.1 MB


In [18]:
orders_deliver_time_visual.head()

,order_approved_at,real_delivery_time
0,2017-10-02 11:07:15,8
1,2018-07-26 03:24:27,12
2,2018-08-08 08:55:23,9
3,2017-11-18 19:45:59,13
4,2018-02-13 22:20:29,2


- 그냥 따라서 가보자

In [20]:
orders_deliver_time_visual = orders_deliver_time[['order_approved_at', 'real_delivery_time']].copy()
orders_deliver_time_visual['real_delivery_time'] = orders_deliver_time_visual['real_delivery_time'].dt.total_seconds()
orders_deliver_time_visual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91638 entries, 0 to 99440
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_approved_at   91638 non-null  datetime64[ns]
 1   real_delivery_time  91638 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.1 MB


In [21]:
orders_deliver_time_visual['real_delivery_time'] = orders_deliver_time_visual['real_delivery_time']/86400
orders_deliver_time_visual.set_index('order_approved_at')
orders_deliver_time_visual.head()

,order_approved_at,real_delivery_time
0,2017-10-02 11:07:15,8.429144
1,2018-07-26 03:24:27,12.502292
2,2018-08-08 08:55:23,9.382708
3,2017-11-18 19:45:59,13.196331
4,2018-02-13 22:20:29,2.830938


- ㄹㅇ 월별

In [22]:
orders_deliver_time_visual.groupby(pd.Grouper(freq = 'M')).mean()

TypeError: Only valid with DatetimeIndex, TimedeltaIndex or PeriodIndex, but got an instance of 'Int64Index'

In [23]:
orders_deliver_time_visual.index

Int64Index([    0,     1,     2,     3,     4,     5,     7,     8,     9,
               10,
            ...
            99430, 99431, 99433, 99434, 99435, 99436, 99437, 99438, 99439,
            99440],
           dtype='int64', length=91638)

#### set_index를 저장을 안했네

In [25]:
orders_deliver_time_visual = orders_deliver_time[['order_approved_at', 'real_delivery_time']].copy()
orders_deliver_time_visual['real_delivery_time'] = orders_deliver_time_visual['real_delivery_time'].dt.total_seconds()
orders_deliver_time_visual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91638 entries, 0 to 99440
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   order_approved_at   91638 non-null  datetime64[ns]
 1   real_delivery_time  91638 non-null  float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 2.1 MB


In [26]:
orders_deliver_time_visual['real_delivery_time'] = orders_deliver_time_visual['real_delivery_time']/86400
orders_deliver_time_visual = orders_deliver_time_visual.set_index('order_approved_at')
orders_deliver_time_visual.head()

,real_delivery_time
order_approved_at,
2017-10-02 11:07:15,8.429144
2018-07-26 03:24:27,12.502292
2018-08-08 08:55:23,9.382708
2017-11-18 19:45:59,13.196331
2018-02-13 22:20:29,2.830938


In [31]:
orders_deliver_time_visual_month = orders_deliver_time_visual.groupby(pd.Grouper(freq = 'M')).mean()
orders_deliver_time_visual_month.head()

,real_delivery_time
order_approved_at,
2016-10-31,14.302643
2016-11-30,NaN
2016-12-31,4.682720
2017-01-31,10.542122
2017-02-28,11.368190


In [33]:
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected = True)

orders_deliver_time_visual_month.iplot(kind = 'bar')

In [34]:
orders_deliver_time_visual_month = orders_deliver_time_visual_month[orders_deliver_time_visual_month.index >= '2017-01-01']

In [37]:
layout = {
    'xaxis':{
        'showticklabels':True,
        'dtick':'M1',
        'tick0':'2017-01',
        'tickfont':{
            # 'tick'
            'size':7
        }
    }
}

orders_deliver_time_visual_month.iplot(kind = 'bar', layout=layout)

In [38]:
layout = {
    'xaxis':{
        'showticklabels':True,
        'dtick':'M1',
        'tick0':'2017-01-31',
        #                01 X
        'tickfont':{
            'size':7
        }
    }
}

orders_deliver_time_visual_month.iplot(kind = 'bar', layout=layout)

In [43]:
colors = ['#BF2C47'] *len(orders_deliver_time_visual_month.index)
for index in range(0,14):
    #              [1:14]로 했더니 에러
    colors[index] = '#1B80BF'

#### range 문법, []슬래싱  다시 익히기

In [48]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x= orders_deliver_time_visual_month.index,
        y= orders_deliver_time_visual_month['real_delivery_time'],
        text= orders_deliver_time_visual_month['real_delivery_time'],
        textposition= 'auto',
        texttemplate= '%{text:.2f} days',
        marker_color = colors
    )
)

fig.update_layout(
    {
        'title':{
            'text':'Mean of Delivery time per Month',
            'x':0.5,
            'y':0.9
        },
        'xaxis':{
            'title':'from Jan.2017 to Aug.2018',
            'showticklabels':True,
            'dtick':'M1',
            'tick0':'2017-01-31',
        },
        'yaxis':{
            'title':'Delivery Time(Days)'
        },
        'template':'plotly_white'
        #           plotly.white
    }
)

fig.show()

In [58]:
colors = ['#BF2C47'] *len(orders_deliver_time_visual_month.index)
for index in range(0,15):
    #                 ㄴ 마지막 번호 포함안됨\
    colors[index] = '#1B80BF'

In [61]:
import plotly.graph_objs as go

fig = go.Figure()
fig.add_trace(
    go.Bar(
        x= orders_deliver_time_visual_month.index,
        y= orders_deliver_time_visual_month['real_delivery_time'],
        text= orders_deliver_time_visual_month['real_delivery_time'],
        textposition= 'auto',
        texttemplate= '%{text:.2f} days',
        marker_color = colors
    )
)

fig.update_layout(
    {
        'title':{
            'text':'<b>Mean of Delivery time per Month</b>',
            'x':0.5,
            'y':0.9,
            'font':{
                'size':15
            }
        },
        'xaxis':{
            'title':'<b>from Jan.2017 to Aug.2018</b>',
            'showticklabels':True,
            'dtick':'M1',
            'tick0':'2017-01-31',
            'tickfont':{
                'size':7
            }
        },
        'yaxis':{
            'title':'Delivery Time(Days)'
        },
        'template':'plotly_white'
        #           plotly.white
    }
)

fig.add_annotation(
    x= '2018-04-30',
    y= orders_deliver_time_visual_month['real_delivery_time'].loc['2018-04-30'],
    #                                       열               iloc
    text= '<b>Imporved Logistic system</b>',
    font= dict(
        size= 10,
        color= "#ffffff"
        #    :
    ),
    # {} 안되네
    align= 'center',
    
    showarrow= True,
    arrowhead= 2,
    arrowsize= 1,
    arrowwidth= 2,
    arrowcolor= '#77BDD9',
    
    bordercolor= '#77BDD9',
    borderwidth= 2,
    borderpad= 2,
    
    ax=40,
    ay=-40,
    
    bgcolor='#F22E62',
    opacity=0.9
)

fig.show()